# 📈 Evaluación del Modelo Final (Métricas y Gráficos)

## 🎯 Objetivo
En este notebook evaluamos el rendimiento del modelo seleccionado (`best_pipeline.pkl`) utilizando datos no vistos (o un subconjunto de validación).
Analizamos métricas clave para clasificación binaria en el contexto médico.

## 📊 Métricas Principales
- **Matriz de Confusión**: ¿Cuántos enfermos detectamos correctamente (TP) y cuántos sanos alarmamos falsamente (FP)?
- **Recall (Sensibilidad)**: Capacidad del modelo para identificar positivos. Es nuestra prioridad.
- **Precision**: De los que el modelo dice que están enfermos, ¿cuántos lo están realmente?
- **F1-Score**: Balance armónico entre Precision y Recall.
- **AUC-ROC**: Capacidad discriminante global del modelo.

## 🔍 Interpretabilidad
- **Feature Importance**: ¿Qué biomarcadores (Edad, Glucosa, Presión) influyen más en la predicción?

In [ ]:
import pandas as pd
from pycaret.classification import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay
import os
import json

# ==========================================
# CONFIGURATION
# ==========================================
MODEL_PATH = "../models/best_pipeline"
DATA_PATH = "../data/02_intermediate/process_data.parquet"
CONFIG_PATH = "../models/model_config.json"


## 1. Carga del Modelo y Datos de Prueba

In [ ]:
# ==========================================
# 1. LOAD MODEL & DATA
# ==========================================
pipeline = load_model(MODEL_PATH)

# Load Data for Evaluation
df = pd.read_parquet(DATA_PATH)

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

# Use a sample for quick evaluation if dataset is huge
df_eval = df.sample(frac=0.2, random_state=123) 
print(f"Evaluation Data Shape: {df_eval.shape}")


## 2. Generación de Predicciones
Aplicamos el modelo sobre el set de evaluación para obtener etiquetas y probabilidades.

In [ ]:
# ==========================================
# 2. GENERATE PREDICTIONS
# ==========================================
predictions = predict_model(pipeline, data=df_eval)
# PyCaret appends 'prediction_label' and 'prediction_score'
print(predictions.head())


## 3. Análisis de Errores (Matriz de Confusión)
Visualizamos la distribución de aciertos y fallos. Nos interesa minimizar los Falsos Negativos (pacientes enfermos diagnosticados como sanos).

In [ ]:
# ==========================================
# 3. CONFUSION MATRIX
# ==========================================
y_true = predictions[config['target']]
y_pred = predictions['prediction_label']

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()


## 4. Métricas de Desempeño
- **Recall**: Crítico para tamizaje médico.
- **AUC**: Medida de separabilidad entre clases.

In [ ]:
# ==========================================
# 4. METRICS REPORT
# ==========================================
print("Classification Report:")
print(classification_report(y_true, y_pred))

try:
    # Calculate AUC if scores are available
    # prediction_score is the probability of the predicted class. 
    # For AUC we need probability of the positive class.
    # PyCaret's prediction_score is max(prob_0, prob_1). 
    # We assume binary classification 0/1.
    # If label is 1, prob_1 = score. If label is 0, prob_1 = 1 - score.
    
    probs = predictions.apply(lambda x: x['prediction_score'] if x['prediction_label'] == 1 else 1 - x['prediction_score'], axis=1)
    auc = roc_auc_score(y_true, probs)
    print(f"ROC AUC Score: {auc:.4f}")
except Exception as e:
    print(f"AUC Calculation Warning: {e}")


## 5. Explicabilidad del Modelo
Identificamos los factores de riesgo más importantes según el modelo aprendido.

In [ ]:
# ==========================================
# 5. FEATURE IMPORTANCE
# ==========================================
# Attempt to plot feature importance using PyCaret or extraction
try:
    plot_model(pipeline, plot='feature')
except:
    print("PyCaret plot_model failed or not supported for this pipeline. Attempting manual plot.")
    try:
        # Extract model from pipeline (usually the last step)
        model_step = pipeline.steps[-1][1]
        if hasattr(model_step, 'feature_importances_'):
            importances = pd.Series(model_step.feature_importances_, index=config['features'])
            importances.nlargest(10).plot(kind='barh', title='Top 10 Feature Importances')
            plt.show()
    except Exception as e:
        print(f"Manual feature importance plot failed: {e}")
